# FOURSQUARE ASSIGNMENT

## Description    
Did you heard about Hallyu ? It means 'Korean Wave'.  
One of Korean Wave, foreign visitors frequent increangly.   
I work for J.W Mriott Dong-Dae-Moon in KOREA. (not really)     
I would like to recommend good restaurant nearby our hotel for our foreign guest.   

## Gathering Data
To recommend good restaurant for our guest,  
I am going to gather restaurant information around J.W Mariott Dong-Dae-Moon using <a href="https://developer.foursquare.com/">FOURSQUARE API</a>  
From FOURSQUARE API, I can get information like restaurantname, streetname, detail address(latitude and longitude) about restaurant. 

To achieve this, using major python libraries like below.  
1. requests
1. pandas
1. numpy
1. folium
1. geocoders

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

In [2]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190408'
LIMIT = 30

In [3]:
address = '종로구 청계천로 279'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

search_query = '식당'
radius = 1000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [4]:
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",False,4cc8fee9786e4688179c9109,NaN,KR,NaN,대한민국,NaN,244,"[광희동, 대한민국]","[{'label': 'display', 'lat': 37.567874, 'lng':...",37.567874,127.011671,NaN,NaN,NaN,보원 식당,v-1554689097
1,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",False,4de8c5251838b9ad8d15f5c1,NaN,KR,NaN,대한민국,NaN,865,"[숭인1동, 대한민국]","[{'label': 'display', 'lat': 37.57740002525583...",37.577400,127.015376,NaN,NaN,NaN,청포도 식당,v-1554689097
2,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",False,4f436d18e4b09d14bc73f79c,장충단로8길 45,KR,중구,대한민국,NaN,930,"[장충단로8길 45, 장충동, 중구, 경기도, 대한민국]","[{'label': 'display', 'lat': 37.562265, 'lng':...",37.562265,127.008423,NaN,NaN,경기도,서씨네 식당,v-1554689097
3,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,50b88032e4b026fe4a2753ee,NaN,KR,NaN,대한민국,NaN,874,"[광희동, 대한민국]","[{'label': 'display', 'lat': 37.56476331656288...",37.564763,127.005162,NaN,NaN,NaN,깔링까 러시아 식당,v-1554689097
4,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",False,4f7d60e5e4b09d3099e463d7,NaN,KR,NaN,대한민국,NaN,941,"[신당1동, 대한민국]","[{'label': 'display', 'lat': 37.561636, 'lng':...",37.561636,127.010536,NaN,NaN,NaN,가마솥 식당,v-1554689097
